# 第18章：データベース連携（PDO）解答例

各課題の解答例を以下に示します。実践的なWeb開発でよく使用されるパターンを意識して実装しています。

## 課題1：データベース接続クラスの作成 - 解答例

In [ ]:
<?php
// config/database.php
return [
    'host' => 'localhost',
    'dbname' => 'php_learning',
    'username' => 'root',
    'password' => '',
    'charset' => 'utf8mb4',
    'options' => [
        PDO::ATTR_ERRMODE => PDO::ERRMODE_EXCEPTION,
        PDO::ATTR_DEFAULT_FETCH_MODE => PDO::FETCH_ASSOC,
        PDO::ATTR_EMULATE_PREPARES => false,
    ]
];

<?php
class DatabaseConnector {
    private static $instance = null;
    private $pdo;
    
    // プライベートコンストラクタ
    private function __construct() {
        try {
            $config = require __DIR__ . '/config/database.php';
            $dsn = "mysql:host={$config['host']};dbname={$config['dbname']};charset={$config['charset']}";
            $this->pdo = new PDO($dsn, $config['username'], $config['password'], $config['options']);
        } catch (PDOException $e) {
            // 実際のアプリケーションではここでログ出力を行う
            throw new DatabaseException("データベース接続に失敗しました: " . $e->getMessage());
        }
    }
    
    // インスタンス取得メソッド
    public static function getInstance() {
        if (self::$instance === null) {
            self::$instance = new self();
        }
        return self::$instance;
    }
    
    // PDO接続を取得
    public function getConnection() {
        return $this->pdo;
    }
    
    // クローン禁止
    public function __clone() {
        throw new RuntimeException("クローニングは許可されていません");
    }
    
    // セリアライズ禁止
    public function __sleep() {
        throw new RuntimeException("シリアライズは許可されていません");
    }
}

## 課題2：ユーザー管理システムのCRUD操作 - 解答例

In [ ]:
<?php
require_once 'DatabaseConnector.php';

class User {
    private $db;
    
    public function __construct(DatabaseConnector $db) {
        $this->db = $db->getConnection();
    }
    
    /**
     * ユーザーを作成
     */
    public function createUser($name, $email, $password) {
        try {
            $hashedPassword = password_hash($password, PASSWORD_DEFAULT);
            $sql = "INSERT INTO users (name, email, password) VALUES (:name, :email, :password)";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':name', $name);
            $stmt->bindParam(':email', $email);
            $stmt->bindParam(':password', $hashedPassword);
            $stmt->execute();
            return $this->db->lastInsertId();
        } catch (PDOException $e) {
            if ($e->getCode() == 23000) {
                throw new RuntimeException('このメールアドレスは既に使用されています');
            }
            throw new DatabaseException('ユーザー作成に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * すべてのユーザーを取得
     */
    public function getAllUsers() {
        try {
            $sql = "SELECT * FROM users ORDER BY created_at DESC";
            $stmt = $this->db->query($sql);
            return $stmt->fetchAll();
        } catch (PDOException $e) {
            throw new DatabaseException('ユーザー取得に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * IDでユーザーを取得
     */
    public function getUserById($id) {
        try {
            $sql = "SELECT * FROM users WHERE id = :id";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':id', $id, PDO::PARAM_INT);
            $stmt->execute();
            $user = $stmt->fetch();
            
            if (!$user) {
                throw new UserNotFoundException("ID {$id} のユーザーが見つかりません");
            }
            
            return $user;
        } catch (PDOException $e) {
            throw new DatabaseException('ユーザー取得に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * ユーザー情報を更新
     */
    public function updateUser($id, $name, $email, $password = null) {
        try {
            $sql = "UPDATE users SET name = :name, email = :email";
            $params = [':name' => $name, ':email' => $email, ':id' => $id];
            
            if ($password) {
                $hashedPassword = password_hash($password, PASSWORD_DEFAULT);
                $sql .= ", password = :password";
                $params[':password'] = $hashedPassword;
            }
            
            $sql .= " WHERE id = :id";
            $stmt = $this->db->prepare($sql);
            
            foreach ($params as $key => $value) {
                $stmt->bindValue($key, $value, $key === ':id' ? PDO::PARAM_INT : PDO::PARAM_STR);
            }
            
            $stmt->execute();
            return $stmt->rowCount() > 0;
        } catch (PDOException $e) {
            if ($e->getCode() == 23000) {
                throw new RuntimeException('このメールアドレスは既に使用されています');
            }
            throw new DatabaseException('ユーザー更新に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * ユーザーを削除
     */
    public function deleteUser($id) {
        try {
            $sql = "DELETE FROM users WHERE id = :id";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':id', $id, PDO::PARAM_INT);
            $stmt->execute();
            return $stmt->rowCount() > 0;
        } catch (PDOException $e) {
            throw new DatabaseException('ユーザー削除に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * Eメールでユーザーを検索
     */
    public function getUserByEmail($email) {
        try {
            $sql = "SELECT * FROM users WHERE email = :email";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':email', $email);
            $stmt->execute();
            $user = $stmt->fetch();
            
            if (!$user) {
                throw new UserNotFoundException("メール {$email} のユーザーが見つかりません");
            }
            
            return $user;
        } catch (PDOException $e) {
            throw new DatabaseException('ユーザー検索に失敗しました: ' . $e->getMessage());
        }
    }
}

## 課題3：トランザクション処理の実装 - 解答例

In [ ]:
<?php
require_once 'DatabaseConnector.php';

class Account {
    private $db;
    
    public function __construct(DatabaseConnector $db) {
        $this->db = $db->getConnection();
    }
    
    /**
     * 口座を作成
     */
    public function createAccount($userId, $initialBalance = 0) {
        try {
            $sql = "INSERT INTO accounts (user_id, balance) VALUES (:user_id, :balance)";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':user_id', $userId, PDO::PARAM_INT);
            $stmt->bindParam(':balance', $initialBalance, PDO::PARAM_STR);
            $stmt->execute();
            return $this->db->lastInsertId();
        } catch (PDOException $e) {
            throw new DatabaseException('口座作成に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * 口座残高を取得
     */
    public function getBalance($accountId) {
        try {
            $sql = "SELECT balance FROM accounts WHERE id = :id";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':id', $accountId, PDO::PARAM_INT);
            $stmt->execute();
            $result = $stmt->fetch();
            
            if (!$result) {
                throw new RuntimeException("口座 {$accountId} が見つかりません");
            }
            
            return $result['balance'];
        } catch (PDOException $e) {
            throw new DatabaseException('残高取得に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * 残高移動を行う
     */
    public function transferBalance($fromAccountId, $toAccountId, $amount) {
        // トランザクション開始
        $this->db->beginTransaction();
        
        try {
            // 1. 残高チェック
            $fromBalance = $this->getBalance($fromAccountId);
            if ($fromBalance < $amount) {
                throw new RuntimeException('残高不足です');
            }
            
            // 2. 送金元の残高を更新
            $sql = "UPDATE accounts SET balance = balance - :amount WHERE id = :from_id";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':amount', $amount, PDO::PARAM_STR);
            $stmt->bindParam(':from_id', $fromAccountId, PDO::PARAM_INT);
            $stmt->execute();
            
            // 3. 送金先の残高を更新
            $sql = "UPDATE accounts SET balance = balance + :amount WHERE id = :to_id";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':amount', $amount, PDO::PARAM_STR);
            $stmt->bindParam(':to_id', $toAccountId, PDO::PARAM_INT);
            $stmt->execute();
            
            // 4. 履歴を記録（履歴テーブルがある場合）
            // $this->recordTransferHistory($fromAccountId, $toAccountId, $amount);
            
            // 5. ログ出力
            $logMessage = sprintf(
                '残高移動: %s -> %s, 金額: %s',
                $fromAccountId,
                $toAccountId,
                number_format($amount)
            );
            $this->logTransaction($logMessage);
            
            // コミット
            $this->db->commit();
            return true;
            
        } catch (Exception $e) {
            // ロールバック
            $this->db->rollBack();
            
            // エラーログ出力
            $errorMessage = sprintf(
                '残高移動失敗: %s -> %s, 金額: %s, エラー: %s',
                $fromAccountId,
                $toAccountId,
                number_format($amount),
                $e->getMessage()
            );
            $this->logTransaction($errorMessage);
            
            // 例外を再スロー
            throw $e;
        }
    }
    
    /**
     * 取引をログ出力
     */
    private function logTransaction($message) {
        $logFile = __DIR__ . '/../logs/transaction.log';
        $timestamp = date('Y-m-d H:i:s');
        $logEntry = "[{$timestamp}] {$message}\n";
        
        // ディレクトリが存在しない場合は作成
        if (!file_exists(dirname($logFile))) {
            mkdir(dirname($logFile), 0755, true);
        }
        
        file_put_contents($logFile, $logEntry, FILE_APPEND);
    }
}

## 課題4：検索機能の高度化 - 解答例

In [ ]:
<?php
class User {
    // ... 前のメソッド ...
    
    /**
     * 高度なユーザー検索
     */
    public function searchUsers($conditions = [], $limit = 10, $offset = 0) {
        try {
            // 基本クエリ
            $sql = "SELECT * FROM users WHERE 1=1";
            $params = [];
            
            // 名前で前方一致検索
            if (!empty($conditions['name'])) {
                $sql .= " AND name LIKE :name";
                $params[':name'] = $conditions['name'] . '%';
            }
            
            // Eメールで部分一致検索
            if (!empty($conditions['email'])) {
                $sql .= " AND email LIKE :email";
                $params[':email'] = '%' . $conditions['email'] . '%';
            }
            
            // 作成日範囲で検索
            if (!empty($conditions['created_from'])) {
                $sql .= " AND created_at >= :created_from";
                $params[':created_from'] = $conditions['created_from'];
            }
            
            if (!empty($conditions['created_to'])) {
                $sql .= " AND created_at <= :created_to";
                $params[':created_to'] = $conditions['created_to'];
            }
            
            // ソート（デフォルトは作成日順）
            $sql .= " ORDER BY created_at DESC";
            
            // ページネーション
            $sql .= " LIMIT :limit OFFSET :offset";
            $params[':limit'] = $limit;
            $params[':offset'] = $offset;
            
            // プリペアドステートメントを実行
            $stmt = $this->db->prepare($sql);
            
            // パラメータをバインド
            foreach ($params as $key => $value) {
                if ($key === ':limit' || $key === ':offset') {
                    $stmt->bindValue($key, $value, PDO::PARAM_INT);
                } else {
                    $stmt->bindValue($key, $value);
                }
            }
            
            $stmt->execute();
            $users = $stmt->fetchAll();
            
            // 総数取得（ページネーションのために必要）
            $countSql = preg_replace('/SELECT \* FROM/', 'SELECT COUNT(*) FROM', $sql);
            $countSql = preg_replace('/ LIMIT :limit OFFSET :offset/', '', $countSql);
            
            $countStmt = $this->db->prepare($countSql);
            foreach ($params as $key => $value) {
                if ($key === ':limit' || $key === ':offset') {
                    continue;
                }
                if ($key === ':created_from' || $key === ':created_to') {
                    $countStmt->bindValue($key, $value, PDO::PARAM_STR);
                } else {
                    $countStmt->bindValue($key, $value);
                }
            }
            $countStmt->execute();
            $totalCount = $countStmt->fetchColumn();
            
            return [
                'users' => $users,
                'total' => $totalCount,
                'pages' => ceil($totalCount / $limit),
                'current_page' => floor($offset / $limit) + 1
            ];
            
        } catch (PDOException $e) {
            throw new DatabaseException('ユーザー検索に失敗しました: ' . $e->getMessage());
        }
    }
}

## 課題5：エラーハンドリングとログ出力 - 解答例

In [ ]:
<?php
// カスタム例外クラス
class UserNotFoundException extends Exception {
    public function __construct($message = "ユーザーが見つかりません", $code = 0, Throwable $previous = null) {
        parent::__construct($message, $code, $previous);
    }
}

class DatabaseException extends Exception {
    public function __construct($message = "データベースエラーが発生しました", $code = 0, Throwable $previous = null) {
        parent::__construct($message, $code, $previous);
    }
}

// ログクラス
class Logger {
    private $logDir;
    
    public function __construct($logDir = __DIR__ . '/../logs') {
        $this->logDir = $logDir;
        if (!file_exists($this->logDir)) {
            mkdir($this->logDir, 0755, true);
        }
    }
    
    /**
     * エラーログを出力
     */
    public function logError(Exception $e, $context = []) {
        $timestamp = date('Y-m-d H:i:s');
        $logEntry = "[{$timestamp}] ERROR: {$e->getMessage()}
";
        $logEntry .= "File: {$e->getFile()} (Line: {$e->getLine()})\n";
        $logEntry .= "Stack Trace:\n{$e->getTraceAsString()}\n";
        
        // コンテキスト情報があれば追加
        if (!empty($context)) {
            $logEntry .= "Context: " . json_encode($context, JSON_UNESCAPED_UNICODE) . "\n";
        }
        
        $logFile = $this->logDir . '/error_' . date('Ymd') . '.log';
        file_put_contents($logFile, $logEntry, FILE_APPEND);
    }
    
    /**
     * 一般ログを出力
     */
    public function logInfo($message, $context = []) {
        $timestamp = date('Y-m-d H:i:s');
        $logEntry = "[{$timestamp}] INFO: {$message}\n";
        
        if (!empty($context)) {
            $logEntry .= "Context: " . json_encode($context, JSON_UNESCAPED_UNICODE) . "\n";
        }
        
        $logFile = $this->logDir . '/app_' . date('Ymd') . '.log';
        file_put_contents($logFile, $logEntry, FILE_APPEND);
    }
}

// Userクラスの修正版
class User {
    private $db;
    private $logger;
    
    public function __construct(DatabaseConnector $db) {
        $this->db = $db->getConnection();
        $this->logger = new Logger();
    }
    
    public function getUserById($id) {
        try {
            $sql = "SELECT * FROM users WHERE id = :id";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':id', $id, PDO::PARAM_INT);
            $stmt->execute();
            $user = $stmt->fetch();
            
            if (!$user) {
                $e = new UserNotFoundException("ID {$id} のユーザーが見つかりません");
                $this->logger->logError($e, ['user_id' => $id]);
                throw $e;
            }
            
            $this->logger->logInfo("ユーザー取得成功", ['user_id' => $id]);
            return $user;
            
        } catch (PDOException $e) {
            $dbException = new DatabaseException('ユーザー取得に失敗しました: ' . $e->getMessage());
            $this->logger->logError($dbException, ['user_id' => $id]);
            throw $dbException;
        }
    }
}

## 発展課題：リレーションシップの実装 - 解答例

In [ ]:
<?php
require_once 'DatabaseConnector.php';

class Post {
    private $db;
    
    public function __construct(DatabaseConnector $db) {
        $this->db = $db->getConnection();
    }
    
    /**
     * 投稿を作成
     */
    public function createPost($userId, $title, $content) {
        try {
            // ユーザーの存在確認
            $userSql = "SELECT id FROM users WHERE id = :id";
            $userStmt = $this->db->prepare($userSql);
            $userStmt->bindParam(':id', $userId, PDO::PARAM_INT);
            $userStmt->execute();
            
            if (!$userStmt->fetch()) {
                throw new UserNotFoundException("ユーザーID {$userId} が存在しません");
            }
            
            $sql = "INSERT INTO posts (user_id, title, content) VALUES (:user_id, :title, :content)";
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':user_id', $userId, PDO::PARAM_INT);
            $stmt->bindParam(':title', $title);
            $stmt->bindParam(':content', $content);
            $stmt->execute();
            
            return $this->db->lastInsertId();
        } catch (PDOException $e) {
            throw new DatabaseException('投稿作成に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * ユーザーのすべての投稿を取得
     */
    public function getUserPosts($userId, $limit = 10, $offset = 0) {
        try {
            // ユーザーの存在確認
            $userSql = "SELECT id FROM users WHERE id = :id";
            $userStmt = $this->db->prepare($userSql);
            $userStmt->bindParam(':id', $userId, PDO::PARAM_INT);
            $userStmt->execute();
            
            if (!$userStmt->fetch()) {
                throw new UserNotFoundException("ユーザーID {$userId} が存在しません");
            }
            
            $sql = "SELECT p.*, u.name as author_name FROM posts p ";
            $sql .= "LEFT JOIN users u ON p.user_id = u.id ";
            $sql .= "WHERE p.user_id = :user_id ";
            $sql .= "ORDER BY p.created_at DESC ";
            $sql .= "LIMIT :limit OFFSET :offset";
            
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':user_id', $userId, PDO::PARAM_INT);
            $stmt->bindParam(':limit', $limit, PDO::PARAM_INT);
            $stmt->bindParam(':offset', $offset, PDO::PARAM_INT);
            $stmt->execute();
            
            return $stmt->fetchAll();
        } catch (PDOException $e) {
            throw new DatabaseException('投稿取得に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * 投稿に属するユーザー情報を取得
     */
    public function getPostWithUser($postId) {
        try {
            $sql = "SELECT p.*, u.name as author_name, u.email as author_email ";
            $sql .= "FROM posts p ";
            $sql .= "LEFT JOIN users u ON p.user_id = u.id ";
            $sql .= "WHERE p.id = :id";
            
            $stmt = $this->db->prepare($sql);
            $stmt->bindParam(':id', $postId, PDO::PARAM_INT);
            $stmt->execute();
            
            $post = $stmt->fetch();
            
            if (!$post) {
                throw new RuntimeException("投稿ID {$postId} が存在しません");
            }
            
            return $post;
        } catch (PDOException $e) {
            throw new DatabaseException('投稿取得に失敗しました: ' . $e->getMessage());
        }
    }
    
    /**
     * 投稿を削除
     */
    public function deletePost($postId) {
        // トランザクションを使用して関連データも削除
        $this->db->beginTransaction();
        
        try {
            // コメントがあれば削除（commentsテーブルがある場合）
            $deleteCommentsSql = "DELETE FROM comments WHERE post_id = :post_id";
            $deleteCommentsStmt = $this->db->prepare($deleteCommentsSql);
            $deleteCommentsStmt->bindParam(':post_id', $postId, PDO::PARAM_INT);
            $deleteCommentsStmt->execute();
            
            // 投稿を削除
            $deletePostSql = "DELETE FROM posts WHERE id = :id";
            $deletePostStmt = $this->db->prepare($deletePostSql);
            $deletePostStmt->bindParam(':id', $postId, PDO::PARAM_INT);
            $deletePostStmt->execute();
            
            if ($deletePostStmt->rowCount() === 0) {
                throw new RuntimeException("投稿ID {$postId} が存在しません");
            }
            
            $this->db->commit();
            return true;
            
        } catch (Exception $e) {
            $this->db->rollBack();
            throw $e;
        }
    }
}

## 使用例

以下はこれらのクラスを使用する基本的な例です：

In [ ]:
<?php
require_once 'DatabaseConnector.php';
require_once 'User.php';
require_once 'Account.php';
require_once 'Post.php';
require_once 'Logger.php';

// データベース接続
try {
    $db = DatabaseConnector::getInstance();
    $user = new User($db);
    $account = new Account($db);
    $post = new Post($db);
    
    // ユーザー作成
    $userId = $user->createUser('山田太郎', 'yamada@example.com', 'password123');
    echo "ユーザーを作成しました: ID {$userId}\n";
    
    // 口座作成
    $accountId = $account->createAccount($userId, 10000);
    echo "口座を作成しました: ID {$accountId}\n";
    
    // 残高移動
    $account2 = $account->createAccount(2, 5000);
    $account->transferBalance($accountId, $account2, 3000);
    echo "残高を移動しました\n";
    
    // ユーザー検索
    $results = $user->searchUsers(['name' => '山田'], 5, 0);
    echo "検索結果: {$results['total']}件が見つかりました\n";
    
    // 投稿作成
    $postId = $post->createPost($userId, '初投稿', 'これは最初の投稿です');
    echo "投稿を作成しました: ID {$postId}\n";
    
} catch (Exception $e) {
    echo "エラーが発生しました: " . $e->getMessage() . "\n";
    
    // エラーログ出力
    $logger = new Logger();
    $logger->logError($e);
}